# About Florida Insurance Analysis

Book combines historical publicly reported data from the FL department of insurance to evaluate trends in market share, claims frequency/severity, and hurricane losses.

Data downloaded from https://apps.fldfs.com/QSRNG/Reports/ReportCriteria.aspx on 04 and 05 April 2025.

Environment AKStandard (Python 3.11.9)

# Imports

In [30]:
import pandas as pd # data ingestion, manipulation
from os import listdir # importing raw data
import time # I just want to very briefly look at data and have it print for a second to glance at it
from IPython.display import clear_output # thanks to ChatGPT - lets me clear printed data

# Data Extraction/Ingestion

Outline/Plan

function - import, remove grand total\
function - check/remove blank columns\
compile to a single df\
\
future: move to AW S3 bucket, SQL access

In [92]:
# listdir('./data/raw')[-1]

In [93]:
info = pd.read_excel('./data/raw/2009Q1.xlsx', skiprows = 14)
print(f"The imported shape is: {info.shape}\nThe last named insurers are:\n{info.iloc[-2:,1]}\n")
grand_total_row = info.shape[0] - 1
info.drop(grand_total_row, axis=0, inplace=True)
print(f"The updated shape is: {info.shape}\nThe last named insurers are:\n{info.iloc[-2:,1]}")

The imported shape is: (214, 65)
The last named insurers are:
212    TESLA PROPERTY & CASUALTY, INC.
213                                NaN
Name: Insurer Name, dtype: object

The updated shape is: (213, 65)
The last named insurers are:
211    STATE NATIONAL INSURANCE COMPANY INC.
212          TESLA PROPERTY & CASUALTY, INC.
Name: Insurer Name, dtype: object


In [94]:
len(info.columns)

65

In [95]:
test = 'this is a test string thingy line'
if "hi" in test:
    print('hi')

hi


In [96]:
counter = 0
for x in info.columns:
    counter += 1
    clear_output(wait = True)
    print(f"Iteration: {counter} Column: {x}\n", info[x])
    time.sleep(0.2)

Iteration: 65 Column: Unnamed: 64
 0      0%
1      0%
2      0%
3      0%
4      0%
       ..
208    0%
209    0%
210    0%
211    0%
212    0%
Name: Unnamed: 64, Length: 213, dtype: object


In [97]:
info.iloc[:,62]

0      0%
1      0%
2      0%
3      0%
4      0%
       ..
208    0%
209    0%
210    0%
211    0%
212    0%
Name: Unnamed: 62, Length: 213, dtype: object

In [98]:
info.iloc[:,62].isnull().all()

False

In [99]:
x = "Rank"
# info.loc[:,x]
info.loc[:,x].isnull().all()

False

In [100]:
null_columns = []
not_null_columns = []
leftover_columns = []

for x in info.columns:
    if "Unnamed" not in x and info.loc[:,x].isnull().all() == False:
        not_null_columns.append(x)
    elif "Unnamed" in x and info.loc[:,x].isnull().all() == False:
        leftover_columns.append(x)
    else:
        if info.loc[:,x].isnull().all():
            null_columns.append(x)

null_columns
not_null_columns
leftover_columns

['Unnamed: 10',
 'Unnamed: 14',
 'Unnamed: 19',
 'Unnamed: 21',
 'Unnamed: 23',
 'Unnamed: 25',
 'Unnamed: 27',
 'Unnamed: 29',
 'Unnamed: 31',
 'Unnamed: 33',
 'Unnamed: 35',
 'Unnamed: 37',
 'Unnamed: 39',
 'Unnamed: 41',
 'Unnamed: 43',
 'Unnamed: 46',
 'Unnamed: 48',
 'Unnamed: 50',
 'Unnamed: 52',
 'Unnamed: 54',
 'Unnamed: 56',
 'Unnamed: 58',
 'Unnamed: 60',
 'Unnamed: 62',
 'Unnamed: 64']

In [101]:
print(null_columns)
print(not_null_columns)
print(leftover_columns)

['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 9', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 15', 'Unnamed: 17', 'Unnamed: 18', 'Claims Opened', 'Unnamed: 45', 'Claims Closed', 'Claims Pending', 'Claims Alternative Dispute', 'Claims with Mediation', 'Claims with Arbitration', 'Claims with Appraisal', 'Claims with Sink Hole Eval', 'Claims with Settlement Conf', 'Claims with Other']
['Rank', 'Insurer Name', 'Policies In Force', 'Total $ value of exposure for policies in force that include wind coverage', 'Number of policies canceled', 'Number of policies nonrenewed', 'Number of policies canceled due to hurricane risk', 'Number of policies nonrenewed due to hurricane risk', 'Number of new policies written', 'Policies in force that exclude wind coverage', 'Total premiums written', 'Policies in force that include wind coverage', 'Total $ value of exposure for policies in force that exclude wind coverage', 'Direct premium written for policies in 

In [102]:
info[null_columns]

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 9,Unnamed: 12,Unnamed: 13,Unnamed: 15,...,Unnamed: 45,Claims Closed,Claims Pending,Claims Alternative Dispute,Claims with Mediation,Claims with Arbitration,Claims with Appraisal,Claims with Sink Hole Eval,Claims with Settlement Conf,Claims with Other
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
# pandas display options - https://pandas.pydata.org/docs/user_guide/options.html

pd.options.display.max_rows
# pd.options.display.max_columns = 50
pd.reset_option("display.max_columns")

In [ ]:
mod_df = info.copy().drop(labels=null_columns, axis = 1)

mod_df.shape

(213, 42)

In [104]:
print(mod_df)

    Rank                                     Insurer Name  Policies In Force  \
0      1          CITIZENS PROPERTY INSURANCE CORPORATION            1005077   
1      2             STATE FARM FLORIDA INSURANCE COMPANY             853867   
2      3  UNIVERSAL PROPERTY & CASUALTY INSURANCE COMPANY             498139   
3      4                ST. JOHNS INSURANCE COMPANY, INC.             204409   
4      5           UNITED SERVICES AUTOMOBILE ASSOCIATION             163850   
..   ...                                              ...                ...   
208  209              AMERICAN COMMERCE INSURANCE COMPANY                  0   
209  210      AMERICAN GENERAL PROPERTY INS CO OF FLORIDA                  0   
210  211                  EMC PROPERTY & CASUALTY COMPANY                  0   
211  212            STATE NATIONAL INSURANCE COMPANY INC.                  0   
212  213                  TESLA PROPERTY & CASUALTY, INC.                  0   

    Unnamed: 10  \
0        16.20%   
1

In [105]:
info.loc[:,'Unnamed: 35']

0       3.93%
1      11.71%
2       0.68%
3       0.14%
4       4.36%
        ...  
208        0%
209        0%
210        0%
211        0%
212        0%
Name: Unnamed: 35, Length: 213, dtype: object

In [106]:
import time

items = ["Item 1", "Item 2", "Item 3", "Item 4"]

for item in items:
    clear_output(wait = True)
    print(item)
    time.sleep(1)

# Optionally move to a new line after the loop
print()


Item 4



In [107]:
# chatty G getting wild

from IPython.display import display, clear_output
import ipywidgets as widgets

items = ["Apple", "Banana", "Cherry", "Date"]
saved_items = []
index = 0  # Keeps track of which item we're on

# Buttons
save_button = widgets.Button(description="Save")
skip_button = widgets.Button(description="Skip")
output = widgets.Output()

def handle_click(button):
    global index
    if button == save_button:
        saved_items.append(items[index])
    
    index += 1
    if index < len(items):
        show_item()
    else:
        clear_output()
        print("Done! Saved items:", saved_items)

def show_item():
    clear_output(wait=True)
    print("Current item:", items[index])
    display(save_button, skip_button)

save_button.on_click(handle_click)
skip_button.on_click(handle_click)

# Start it off
show_item()


ModuleNotFoundError: No module named 'ipywidgets'